In [3]:
import nibabel as nib 
! pip install nibabel --user
import os
import glob
import numpy as np
from nibabel.testing import data_path
import matplotlib.pyplot as plt
#import cv2
#! pip install cv2 --user
import time
from datetime import datetime
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.ndimage import zoom
import pandas as pd
import os
from skimage.io import imread
from skimage.color import gray2rgb
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import torchvision as tv
from torch.utils.data import (
    Dataset,
    DataLoader,
)
import matplotlib.pyplot as plt
from torch.utils import data as DT

%matplotlib inline

In [2]:
path = 'D:\\Master_courses\\Praktikum\\Datasets\\Mayank\\hecktor_nii\\'
divisor=128.0

files = [f for f in glob.glob(path + "**/*_ct_gtvt.nii.gz", recursive=True)]
labels_lenght=list()
image_list=list()
label_list=list()

for idx,f in enumerate(files):
    SEG_img = nib.load(f)
    SEG_data = SEG_img.get_fdata()
    prev_shape=SEG_data.shape
    new_shape=[divisor/x for x in prev_shape]
    SEG_data = zoom(SEG_data, new_shape,order=1).astype(np.float)
    label_list.append(SEG_data)
    path=f[:-15]+'_pt.nii.gz'
    im=nib.load(path)
    PET_data=im.get_fdata()
    prev_shape=PET_data.shape
    new_shape=[divisor/x for x in prev_shape]
    PET_data = zoom(PET_data, new_shape,order=1).astype(np.float)
    image_list.append(PET_data)
    #print(idx)


In [88]:
len(image_list) # number of patients

202

In [200]:
size = 32 # patch size
stride = 32 # patch stride

all_labels=list()
for seg_img in label_list:
    patches = torch.from_numpy(seg_img).unfold(0, size, stride).unfold(1, size, stride).unfold(2, size, stride)
    reshaped_patches=patches.reshape(-1,size,size,size)
    labels_=torch.sum(reshaped_patches , dim=(1,2,3)).bool().int().numpy()
    all_labels.append(labels_)
all_labels=np.asarray(all_labels).reshape(-1)

In [201]:
all_labels[:64] # labels of one patient

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [202]:
all_PET_patches=list()
for PET_img in image_list:
    patches = torch.from_numpy(PET_img).unfold(0, size, stride).unfold(1, size, stride).unfold(2, size, stride)
    reshaped_patches=patches.reshape(-1,size,size,size).numpy()
    all_PET_patches.append(reshaped_patches)
all_PET_patches=np.asarray(all_PET_patches).reshape(-1,size,size,size)

In [203]:
print (all_PET_patches.shape) #number of patches * pactch dim1 * patch dim2 * patch dim3
patch_mean=np.mean(all_PET_patches , axis=0)
patch_std=np.std(all_PET_patches , axis=0)

(12928, 32, 32, 32)


In [255]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 32
num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 20


In [205]:
class HN_Dataset(Dataset):
    def __init__(self, transform=tv.transforms.Compose(tv.transforms.ToTensor())):
        self.labels = all_labels
        self.images=all_PET_patches
        self.transform = transform
        self.mode='None'
    def __len__(self):
        return len(all_labels)

    def __getitem__(self, idx):

        label= torch.from_numpy(np.asarray(self.labels[idx]))
        
        if self.transform is not None:
            PET_data = self.transform(self.images[idx])
        return (PET_data, label)

In [206]:
split_threshold=12000  # to split test and train set

print ('target train 0/1: {}/{}'.format(
    len(np.where(all_labels[:split_threshold] == 0)[0]), len(np.where(all_labels[:split_threshold] == 1)[0])))

class_sample_count = np.array(
    [len(np.where(all_labels[:split_threshold] == t)[0]) for t in np.unique(all_labels[:split_threshold])])
weight = ([1. ,1.]) / class_sample_count
samples_weight = np.array([weight[t] for t in all_labels[:split_threshold]])
samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight, len(samples_weight))
"""
target = torch.from_numpy(all_labels[:70000]).int()
data=torch.from_numpy(all_PET_patches[:70000])
train_set = torch.utils.data.TensorDataset(data, target)
target = torch.from_numpy(all_labels[70000:]).int()
data=torch.from_numpy(all_PET_patches[70000:])
test_set = torch.utils.data.TensorDataset(data, target)


"""
# Load Data

idx = list(range(len(all_labels)))

dataset = HN_Dataset( transform=tv.transforms.ToTensor())
train_set = DT.Subset(dataset, idx[:split_threshold])
test_set = DT.Subset(dataset, idx[split_threshold:])

#train_set, test_set = torch.utils.data.random_split(dataset, [70000, 6800])

train_set.transform= tv.transforms.Compose([tv.transforms.RandomHorizontalFlip(),
                                            tv.transforms.ToTensor(),
                                            tv.transforms.Normalize(patch_mean, patch_std)])

#tv.transforms.RandomRotation(180), tv.transforms.RandomVerticalFlip(),tv.transforms.RandomVerticalFlip(),

train_loader = DataLoader(train_set, batch_size=batch_size, sampler= sampler)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

for i, (data, target) in enumerate(train_loader):
    print ("batch index {}, 0/1: {}/{}".format(
        i,
        len(np.where(target.numpy() == 0)[0]),
        len(np.where(target.numpy() == 1)[0])))
    #print ((data[1].shape))


target train 0/1: 11595/405
batch index 0, 0/1: 30/34
batch index 1, 0/1: 32/32
batch index 2, 0/1: 34/30
batch index 3, 0/1: 39/25
batch index 4, 0/1: 30/34
batch index 5, 0/1: 29/35
batch index 6, 0/1: 32/32
batch index 7, 0/1: 34/30
batch index 8, 0/1: 28/36
batch index 9, 0/1: 38/26
batch index 10, 0/1: 31/33
batch index 11, 0/1: 30/34
batch index 12, 0/1: 38/26
batch index 13, 0/1: 38/26
batch index 14, 0/1: 36/28
batch index 15, 0/1: 27/37
batch index 16, 0/1: 30/34
batch index 17, 0/1: 36/28
batch index 18, 0/1: 28/36
batch index 19, 0/1: 29/35
batch index 20, 0/1: 31/33
batch index 21, 0/1: 39/25
batch index 22, 0/1: 30/34
batch index 23, 0/1: 32/32
batch index 24, 0/1: 37/27
batch index 25, 0/1: 41/23
batch index 26, 0/1: 34/30
batch index 27, 0/1: 29/35
batch index 28, 0/1: 26/38
batch index 29, 0/1: 40/24
batch index 30, 0/1: 33/31
batch index 31, 0/1: 43/21
batch index 32, 0/1: 33/31
batch index 33, 0/1: 31/33
batch index 34, 0/1: 38/26
batch index 35, 0/1: 34/30
batch inde

In [259]:

dataIter = iter(test_loader)
imgs, labels = dataIter.next()


print ("labels", labels)


labels tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)


In [220]:
class CNN(nn.Module):
    def __init__(self, in_channels=16, num_classes=1):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=(5, 5),
            stride=(1, 1),
            padding=(2, 2),
        )
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=8,
            kernel_size=(3 ,3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.conv3 = nn.Conv2d(
            in_channels=8,
            out_channels=8,
            kernel_size=(3 ,3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.conv4 = nn.Conv2d(
            in_channels=8,
            out_channels=4,
            kernel_size=(3 ,3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.conv5 = nn.Conv2d(
            in_channels=4,
            out_channels=4,
            kernel_size=(3 ,3),
            stride=(1, 1),
            padding=(1, 1),
        )
        self.fc1 = nn.Linear(4 * 2 * 2, num_classes)
        self.sig=nn.Sigmoid()
        self.initialize_weights()

        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        #x = F.relu(self.conv5(x))
        #x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x=self.sig(x)
        return x

    

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)

                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0)


In [221]:

def save_checkpoint(state, filename="my_checkpoint_3D_V2.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])


In [256]:
#device = torch.device("cpu")

# Initialize network
model = CNN().to(device)
#model = model.float()

# Loss and optimizer
criterion = nn.BCELoss()
#criterion=nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
load_module=False
print("Current Time =", datetime.now().strftime("%H:%M:%S")) 
# Train Network
for epoch in range(num_epochs):
    now = datetime.now()
    if load_module:
        # load checkpoint
        load_checkpoint(torch.load("my_checkpoint_3D_V2.pth.tar"), model, optimizer)
        load_module=False
    losses = []
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        data=data.float()
        scores = model(data)
        targets=targets.float()
        scores=scores.view(-1)

        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    if (epoch % 4==0):
        print(f"Cost at epoch {epoch} is {np.sum(losses)/len(losses)}")
        print("Current Time =", now.strftime("%H:%M:%S")) 
        checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
        # save checkpoint
        save_checkpoint(checkpoint)


Current Time = 00:16:35
Cost at epoch 0 is 0.42573357642965115
Current Time = 00:16:35
=> Saving checkpoint
Cost at epoch 4 is 0.13593927494428576
Current Time = 00:17:16
=> Saving checkpoint
Cost at epoch 8 is 0.10998262107332653
Current Time = 00:17:56
=> Saving checkpoint
Cost at epoch 12 is 0.06158317745653318
Current Time = 00:18:37
=> Saving checkpoint
Cost at epoch 16 is 0.056205892229472545
Current Time = 00:19:18
=> Saving checkpoint


In [264]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model, test_set):
    true_positives,false_positives,true_negatives,false_negatives=0,0,0,0

    num_correct = 0
    num_samples = 0
    model.eval()
    j=0

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x=x.float()
            scores = model(x)
            predictions=torch.round(scores).view(-1)
            
            
            #confusion matrix
            confusion_vector = y.cpu() / predictions.cpu()
            true_positives =true_positives+ torch.sum(confusion_vector == 1).item()
            false_positives =false_positives+ torch.sum(confusion_vector == float('inf')).item()
            true_negatives =true_negatives+ torch.sum(torch.isnan(confusion_vector)).item()
            false_negatives =false_negatives+ torch.sum(confusion_vector == 0).item()
            
            
            if test_set:
                #imshow(x.cpu())
                #print ("labels:       ",(y.cpu()))
                #print ("predictions:  ",(predictions.cpu()))
                
                
                print ("labels:       ",np.ravel(y.cpu().numpy()))
                print ("predictions:  ",np.ravel((predictions.cpu().numpy()).astype(np.int)))
                np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
                print (np.ravel(scores.cpu().numpy()))
                #output=scores
                #test_set=False

            num_correct += (predictions.view(-1) == y).sum()
            num_samples += predictions.size(0)
        
        percision= true_positives/(true_positives+false_positives)
        recall=true_positives/(true_positives+false_negatives)
        F1score= 2*(percision*recall)/(percision+recall)
        print(f"Got {num_correct} / {num_samples} with ratio of {float(num_correct)/float(num_samples)*100:.2f}")
        print ("true_positives",true_positives)
        print ("false_positives",false_positives)
        print ("true_negatives",true_negatives)
        print ("false_negatives",false_negatives)
        print ("F1score",F1score)

        
    model.train()



In [265]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model ,test_set=False)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model, test_set=False)

Checking accuracy on Training Set
Got 11876 / 12000 with ratio of 98.97
true_positives 5993
false_positives 0
true_negatives 5883
false_negatives 124
F1score 0.9897605284888522
Checking accuracy on Test Set
Got 893 / 928 with ratio of 96.23
true_positives 30
false_positives 4
true_negatives 863
false_negatives 31
F1score 0.631578947368421


In [263]:

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model, test_set=True)

Checking accuracy on Test Set
labels:        [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictions:   [0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[ 0.001  0.323  0.004  0.000  0.952  0.000  0.999  0.000  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.004  0.000  0.000
  0.003  0.375  0.152  0.000  0.000  0.005  0.000  0.000  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.001  0.005  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.001
  0.000  0.000  0.089  0.881  0.831  0.000  0.000  0.000  0.000  0.000
  0.000  0.000  0.000  0.000]
labels:        [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
predictions:   [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

labels:        [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0]
predictions:   [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
[ 0.787  0.000  0.000  0.000  0.000  0.000  0.839  0.000  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.067  0.002  0.002  0.000
  0.213  0.146  0.562  0.000  0.000  0.000  0.000  0.000  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.002  0.000  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
  0.000  0.000  0.000  0.752  0.981  0.000  0.000  0.000  0.000  0.000
  0.000  0.000  0.000  0.000]
labels:        [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
predictions:   [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0